In [69]:
import math
from scipy.stats import norm

⚠️⚠️⚠️correct for power and correlation?

In [70]:
def sample_size_paired_t(alpha: float, p1: float, p2: float) -> int:
    sigma_d = math.sqrt(p1 * (1 - p1) + p2 * (1 - p2))
    z = norm.ppf(1 - alpha)  # one-sided critical value
    n = (z * sigma_d / (p1 - p2)) ** 2
    return math.ceil(n)


def sample_size_mannwhitney(alpha: float, p1: float, p2: float) -> int:
    z = norm.ppf(1 - alpha / 2)  # two-sided critical value
    n_per_group = (4 * (z**2)) / (3 * (p1 - p2) ** 2)
    total_n = 2 * math.ceil(n_per_group)
    return total_n


def sample_size_chernoff(alpha: float, p1: float, p2: float) -> int:
    kl = p1 * math.log(p1 / p2) + (1 - p1) * math.log((1 - p1) / (1 - p2))
    n = math.log(2 / alpha) / kl
    return math.ceil(n)


def sample_size_hoeffdings(
    alpha: float, p1: float, p2: float, min_val: float = 0.0, max_val: float = 1.0
) -> int:
    R = max_val - min_val
    n = (2 * (R**2) * math.log(2 / alpha)) / ((p1 - p2) ** 2)
    return math.ceil(n)


def mcnemar_size(alpha: float, p1: float, p2: float) -> int:
    z = norm.ppf(1 - alpha / 2)  # two-sided critical value
    p10 = p1 * (1 - p2)
    p01 = p2 * (1 - p1)
    n = (z**2 * (p10 + p01)) / ((p10 - p01) ** 2)
    return math.ceil(n)

In [71]:
alpha = 0.2
p_a = 0.8
p_b = 0.75

In [72]:
n_mcnemar = mcnemar_size(alpha, p_a, p_b)
n_paired_t = sample_size_paired_t(alpha, p_a, p_b)
n_mannwhitney = sample_size_mannwhitney(alpha, p_a, p_b)
n_chernoff = sample_size_chernoff(alpha, p_a, p_b)
n_hoeffdings = sample_size_hoeffdings(alpha, p_a, p_b)

print(f"McNemar: {n_mcnemar}")
print(f"Paired t-test: {n_paired_t}")
print(f"Mann-Whitney: {n_mannwhitney}")
print(f"Chernoff: {n_chernoff}")
print(f"Hoeffding's: {n_hoeffdings}")

McNemar: 230
Paired t-test: 99
Mann-Whitney: 1752
Chernoff: 329
Hoeffding's: 1843


In [73]:
import plotly.express as px
import pandas as pd
import numpy as np

In [74]:
p1 = 0.9
p2 = np.linspace(p1 - 0.1, p1 - 1e-3, 100)
alpha = 0.2

df = pd.DataFrame(
    {
        "p2": p2,
        "McNemar": [mcnemar_size(alpha, p1, p) for p in p2],
        "Paired t-test": [sample_size_paired_t(alpha, p1, p) for p in p2],
        "Mann-Whitney": [sample_size_mannwhitney(alpha, p1, p) for p in p2],
        "Chernoff": [sample_size_chernoff(alpha, p1, p) for p in p2],
        "Hoeffding's": [sample_size_hoeffdings(alpha, p1, p) for p in p2],
    }
)

In [84]:
px.line(
    df,
    x="p2",
    y=["McNemar", "Paired t-test", "Mann-Whitney", "Chernoff", "Hoeffding's"],
    title=f"Sample size vs. p2; p1={p1}",
    range_y=[10, 1000],
    log_y=True,
)

In [99]:
# create graph for paired t-test. x-axis is diff to p1 and the colors are p1=0.9, 0.8, 0.7, 0.6, 0.5

p1 = np.linspace(0.5, 1, 6).round(2)
diff = np.linspace(0.01, 0.1, 100).round(2)
alpha = 0.05

required_n = [sample_size_paired_t(alpha, p, p - d) for p in p1 for d in diff]

df = pd.DataFrame(
    {"p1": np.repeat(p1, 100), "diff": -np.tile(diff, 6), "Paired t-test": required_n}
)

px.line(
    df,
    x="diff",
    y="Paired t-test",
    color="p1",
    title="Sample size vs. difference to p1",
    log_y=True,
    range_y=[10, 1000],
)